In [ ]:
import numpy as np
from PIL import Image
import spacy
import string
import re
import wordcloud
import matplotlib.pyplot as plt
import collections
import multiprocessing
#from unidecode import unidecode
import math

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

import logging  # Setting up the loggings to monitor processes
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

<h1 style="background-color:powderblue;">General helper Classes</h1>